#  Coevolução no jogo Rastros
## Projeto nº 3
### Introdução à Inteligência Artificial - edição 2020/21
#### Entrega: 29 de Novembro (1m antes da meia-noite)

<img src="mobile_relax.gif" alt="Drawing" style="width: 250px;"/>


## Algoritmos coevolucionários

Nos algoritmos evolucionários tradicionais tradicionais o fitness dos indivíduos que fazem parte da população é independente dos outros indivíduos. É o exemplo das N raínhas em que cada elemento da população representa uma dada disposição das N raínhas no tabuleiro e em que o fitness calcula o número de pares de raínhas que não se atacam. Neste caso, o fitness de um indivíduo não depende de modo nenhum das outras disposições das N raínhas correspondentes aos outros indivíduos.

Há algoritmos evolucionários em que existe essa dependência entre os indivíduos da população no cálculo da função de fitness de cada indivíduo. Esse fitness individual pode resultar da cooperação ou competição entre os pares, i.e. entre os indivíduos da população.

### Coevolução para a evolução dos pesos de uma função de avaliação, para uso com o minimax.

No mundo da procura com adversário, em que se usam variantes do algoritmo minimax (o alfabeta, por exemplo) a uma profundidade limitada, para a decisão da jogada a fazer por cada jogador, a função de avaliação estática é fundamental. Em geral, essa função, nos estados não terminais (diferentes de derrota, empate ou vitória) pode ser definida como uma combinação linear de várias características detectadas ou medidas no estado do jogo e dependentes do jogador que está a decidir a sua melhor jogada.

$$linear(jogador,estado) = \sum_{y} P_{y}C_{y}(jogador,estado)$$

Por exemplo, no jogo do Rastros, que todos conhecem por ter sido objecto do segundo projecto, podemos considerar as duas características: distância à base e distância ao objectivo, que estariam implementadas em `distBase` e `distObj`e que a função de avaliação, nos casos em que os jogos não são terminais, é dada por:

$$linear(jogador,estado) = p_{1}*distBase(jogador,estado) + p_{2}*distObj(jogador,estado)$$

A função de avaliação considerando os estados terminais e não terminais, para os pesos $p_{1}$ e $p_{2}$ e para as duas funções características, será equivalente à função seguinte:

```python
def f_eval(jogador,estado):
    if vitória(jogador,estado):
        return infinity
    elif derrota(jogador,estado):
        return -infinity
    else:
        return linear(jogador,estado)
```
Na maior parte dos casos a afinação dos valores dos pesos, fixadas as funções que medem as características, pode ser um processo difícil e demorado de tentativa e erro.

Para evoluir os pesos, podemos utilizar um algoritmo coevolucionário em que os cromossomas são uma cadeia de genes que correspondem à cadeia de pesos e em que a pool de genes pode ser formada por uma gama de inteiros: [inf,sup], indicando os valores possíveis para cada peso. Por exemplo, no caso do exemplo do Rastros, com as duas características referidas em cima, os cromossomas serão cadeias de dois genes e podemos imaginar que a pool de genes pode variar em [-1000,1000]. Um cromossoma ou indivíduo pode ser [12,-200] e nesse caso corresponde ao jogador que usa o alfabeta limitado, com a função de avaliação equivalente à seguinte:

```python
def f_eval(jogador,estado):
    if vitória(jogador,estado):
        return infinity
    elif derrota(jogador,estado):
        return -infinity
    else:
        return 12*distBase(jogador,estado) -200*distObj(jogador,estado)
```

### Cálculo do fitness
O processo deste algoritmo coevolucionário de uma só população (porque existem variantes com mais do que uma população) é de contexto competitivo, porque o cálculo da função de fitness vai depender de uma competição entre os vários indivíduos ou jogadores. Após finalizar a formação da nova geração, realiza-se uma competição entre os vários indivíduos para determinar os respectivos valores de fitnesses.

#### Fitness por round-robin (campeonato)
Essa competição pode ser do tipo "round-robin" em que todos jogam entre si uma série de pares de jogos em que em cada par desempenham os dois papéis: serem os primeiros e segundos a jogar, e em que no final cada um terá como fitness o número de vitórias conseguidas (notem que em jogos em que há empates, é relevante dar um valor para os empates distinto do valor das vitórias e somar a pontuação total).

#### Fitness tipo taça
Neste caso, é necessário que o número de elementos seja uma potência de 2: 2, 4, 8, 16, 32 .... Nesta competição em cada ronda formam-se pares aleatórios de jogadores e cada par de jogadores realiza um duelo entre si. Em cada duelo, os dois jogadores realizam um número fixo de pares de jogos. Num par de jogos entre o jogador 1 e o 2, num dos jogos é o 1 que abre o jogo e no outro é o 2 a abrir. Só os vencedores de cada par é que passam à ronda seguinte. Notem que num duelo entre dois jogadores, as suas pontuações podem ser exactamente iguais, num empaté técnico, e nesse caso o vencedor é escolhido de forma aleatória.

O fitness de cada indivíduo corresponde à ronda final em que abandonam a competição. Por exemplo, se tivermos 8 equipas, isso quer dizer que há 4 jogadores (os perdedores da primeira ronda) terão fitness 1, pois só se aguentam até à primeira ronda, dois deles (os que perderam na segunda ronda) terão fitness 2, um deles terá fitness 3 (perdedor da final) e apenas 1, o vencedor, terá fitness 4. 

O fitness de um indivíduo, nestes dois contextos competitivos, não corresponde a um valor absoluto mas é antes relativo aos adversários. Isso quer dizer que o mesmo jogador em duas populações diferentes pode ter fitnesses diferentes.

### Formação da geração seguinte

#### Selecção/Cruzamento/Mutação
Há variantes que se podem implementar em cada um destes mecanismos:

##### Selecção por torneio de K
O processo de escolha de cada um dos dois cromossomas que se vão cruzar processa-se da seguinte maneira: escolhem-se aleatoriamente K indivíduos da população e selecciona-se o de maior fitness para reprodução. (Este processo é feito para a escolha de cada um dos pais) 

##### Dois descendentes resultam dos cruzamentos
Os cruzamentos de um ponto de corte podem gerar não um (como no tutorial) mas dois descendentes. A parte esquerda do cromossoma do progenitor 1 combina-se com a parte direita do progenitor 2 e a parte esquerda do primeiro progenitor pode combinar-se com a parte direita do segundo.

##### Mutação em todos os genes
Cada um dos genes de um cromossomas pode ser mutado de acordo com a taxa de mutação. Note que no tutorial de genéticos a mutação era feita apenas num dos genes escolhido ao acaso. A mutação de cada um gene, pode ser feita, como no tutorial de genéticos: escolhe-se ao acaso um gene da pool genetica. No entanto, pode-se perturbar o gene somando ou subtrando um valor aleatório delta.

#### Elitismo
Uma forma de tentar manter o melhor indivíduo de uma geração na geração seguinte é implementar um mecanismo de elitismo. Por exemplo, podemos querer passar para a geração seguinte 20% dos melhores indivíduos da geração corrente. Se não houver elitismo, pode acontecer que um jogador ótimo (ou próximo disso) desapareça da evolução para sempre porque mesmo que tenha gerado descendentes, estes podem ter sofrido uma mutação perniciosa. 

### Objetivos

* Queremos que partindo da implementação do algoritmo genético em `geneticSolo.py` desenvolvam um algoritmo coevolucionário capaz de evoluir os pesos de parte de uma função de avaliação dada uma lista de funções características para o jogo do Rastros. Esse algoritmo evolucionário vai usar 

    * fitness tipo taça, em que se realizem jogos entre os jogadores considerando pelo menos 2 limites de profundidade (um par e o outro ímpar);
    * selecção para reprodução através do método de torneio de K, que pode ser de 2 por defeito.
    * cruzamentos num ponto com geração de dois descendentes;
    * mutação em todos os genes (qualquer das opções indicadas em cima (perturbação delta do gene ou substituição por um dos elementos da pool de genes)
    * o mecanismo que permita ter uma percentagem de elitismo. (sugerimos que arredondem para um número par de jogadores de elite)

Notem que os valores de fitness resultam da competição tipo taça em que cada cromossoma corresponde a um jogador que utiliza o alfabeta até uma profundidade limite e em que utilize uma função equivalente à `f_eval` em cima descrita, fazendo uso dos genes. Tenham cuidado de modo a gerarem populações compostas por um número de indivíduos que seja uma potência de 2.  

* Aplicação do algoritmo algoritmo coevolucionário ao jogo Rastros, implementado em `rastros.py` definindo várias funções características. 

### Funções características
No mínimo terão de utilizar as duas funções características descritas em cima. No entanto aconselhamos vivamente que utilizem outras funções características que achem interessantes e que até possam ter usado no 2º projecto.

### Limites da profundidade do alfabeta
É importante que a função de avaliação não dependa do facto de termos limites pares ou ímpares. Reparem que quando o limite de profundidade é ímpar nos estados avaliados é o adversário a jogar a seguir e no caso dos pares é o próprio a jogar a seguir e isso pode fazer a diferença. É conveniente usarem dois ou mais níveis de profundidade (um valor par e um ímpar serão suficientes). Deste modo, por exemplo, para a lista de limites de profundidade [4,5], durante a competição, cada par de jogadores irá realizar um número N (múltiplo de 2) de jogos a um nível de profundidade 4, e depois um número N de jogos a nível 5. O score dos jogadores corresponderá ao número de vitórias nesses 2N jogos.

Podem acelerar a evolução, usando uma profundidade limitada a 2 e a 1, só para testarem o bom funcionamento do algoritmo que executa a coevolução, mas aconselhamos vivamente que tentem evoluir para profundidades maiores depois do algoritmo de coevolução e a integração com o Rastros estiverem em forma.

### Função principal
A função principal deve ter esta assinatura:
    
***coevol(gen,dim,gene_pool,elit,p_mut,k_rivais,lim_profs,n_jogos,caracts)*** em que

    gen:               número de gerações
    dim:               dimensão da população
    gene_pool:         a pool de genes
    elit:              a percentagem da elite
    p_mut:             a probabilidade de mutar cada um dos genes
    k_rivais:          o parâmetro k da selecção por torneio (2 por defeito)
    lim_profs:         a lista de limites de profundidade para o alfabeta
    n_jogos:           o número de pares de jogos para cada limite de profundidade, nos duelos entre jogadores
    caracts:           a lista das funcões que medem as características

## Bónus
Daremos um bónus até 0.25 para os grupos que fizerem a coevolução generalizada a qualquer jogo entre dois jogadores.
Notem que todos as classes dos jogos terão que ter um método `utility` que devolve 1, -1 ou 0 quando o jogo acaba (vitória do primeiro, vitória do segundo, empate, respectivamente). Esse método também devolve 0 quando o jogo ainda não acabou.
Por outro lados, há dois novos atributos em qualquer subclasse de Game, que são `first` e `second`, que indicam respectivamente o jogador que abre o jogo e o outro. No caso do Rastros, é 'S' o `first` e 'N' o `second`; no caso do Galo 'X' é o `first` e 'O' o `second`. 

### Função principal no caso do bónus
A função principal deve ter esta assinatura:
    
***coevol(gen,dim,gene_pool,elit,p_mut,k_rivais,lim_profs,n_jogos,jogo,caracts)*** em que

    gen:               número de gerações
    dim:               dimensão da população
    gene_pool:         a pool de genes
    elit:              a percentagem da elite
    p_mut:             a probabilidade de mutar cada um dos genes
    k_rivais:          o parâmetro k da selecção por torneio (2 por defeito)
    lim_profs:         a lista de limites de profundidade para o alfabeta
    n_jogos:           o número de pares de jogos para cada limite de profundidade, nos duelos entre jogadores
    jogo               a classe do jogo
    caracts:           a lista das funcões que medem as características

### Teste dos programas
Fazendo uso das funcionalidades do notebook que permite a coexistência de texto e código, convém demonstrarem que o algoritmo de coevolução está a brilhar e que a evolução dos pesos da função de avaliação para o jogo Rastros está a funcionar bem.

É importante ilustrar, passo a passo, um conjunto de aspectos em diferentes secções do report:

*Jogadores, Duelos e taças* (com as características de distância à base e distância ao objectivo e a lista de limites de 1 e de 2
* como se constróiem dois jogadores com cromossomas distintos
* a execução de um par de jogos entre os jogadores que correspondem a 2 cromossomas considerando um determinado limite de profundidade de procura.
* os resultados da execução de N pares de jogos entre dois jogadores para cada limite de uma lista de limites e um certo N.
* a formação da geração 0, i.e. a população inicial (dimensão que seja potência de 2) que seja aleatória, para cromossomas para uma determinada pool de genes.
* dada uma população, a execução ronda a ronda, duelo a duelo, de uma competição tipo taça e os valores de fitness que resultam, considerando duelos de N pares de jogos para cada limite.

*Operadores genéticos e elitismo usando populações e cromossomas não apenas com dois pesos*
* a selecção por torneio 
* a geração dos dois descendentes por cruzamento de 1 ponto de corte
* o novo operador de mutação com possibilidade de mutação (em todos os genes)
* a formação da geração seguinte, dada uma geração (considerando a percentagem de elitismo)

*Evolução para limites de 1 e de 2*
* a coevolução no Rastros durante um número de gerações razoável, indicando qual o melhor jogador de cada geração e respectivo fitness, considerando as duas características: distâncias à base e objectivo.

*Evolução para limites de 1 e de 2 para mais características*
* Outras funções características que tenham desenvolvido
* coevolução para limites de 1 e de 2 considerando uma lista maior de características, indicando o melhor jogador de cada geração e respectivo fitness

*Evolução para limites maiores*
* O resultado das evoluções para o Rastros para limites de profundidades maiores  (por exemplo, 4 e 5) indicando quais os parâmetros que usaram, que experiências fizeram.

*Evolução para o Jogo do Galo - extra --- bónus*
* Demonstrem que o algoritmo de coevolução não está feito apenas para o Rastros e que pode ser usado com outros jogos, por exemplo o Jogo do Galo (objecto de uma das PLs) com características que voçês próprios podem inventar. Sugerimos 4 funções características básicas: o número de centros que tenho, o número de centros que ele tem, o número de cantos que tenho, o número de cantos que ele tem. Incluam a classe Galo.py com a modelização do jogo do Galo e suas funções características, no ficheiro onde está o vosso código. 

### Ficheiros incluídos
Os ficheiros incluídos são principalmente:

* `rastros.py` (não alterem nem a classe `Rastros` nem a classe `EstadoRastros`)
* `jogos.py`  (não alterem o algoritmo do alphabeta)
* `utils.py`
* `genetic_tutorial.ipynb`  (tutorial de algoritmos genéticos)
* `geneticSolo.py` (código usado pelo tutorial)
* `genetic_algorithm_example.py`(versão GUI do algoritmo genético em `geneticSolo.py`)

### Entrega
#### Código e Relatório

Todo o código que fizerem deve ser feito num único ficheiro: `IIA2021-proj3-XX.py` (substituam XX pelo número do grupo).
O  relatório é **obrigatório** no formato Jupyter Notebook em `IIA2021-proj3-XX.ipynb`. Qualquer trabalho que não tenha relatório ou que não cumpra esse formato não é avaliado e tem 0 de nota.
Notem que se fizerem o relatório no Jupyter Notebook mas na verdade não fazem nenhum uso das potencialidades do formato notebook, que permite intercalar python e texto, facilitando a demonstração de como funcionam os vários elementos da coevolução,  isso é equivalente a não usarem o formato e a nota será 0!

No notebook devem explicar, ilustrar correndo o código e apresentar os testes que fizeram. Podem inspirar-se nos guiões de IIA ou na resolução do projecto I que está no enunciado da avaliação contínua I.

Não se esqueçam de preencher os nomes e números dos elementos do vosso grupo.

### Prazo
Um zip `IIA2021-proj3-XX.zip` com todo o material incluindo os dois ficheiros `IIA2021-proj3-XX.ipynb` e `IIA2021-proj3-XX.py` até ao dia **29 de Novembro** às 23:59

## Versão 3  (25 de Novembro de 2020)
Alterações da versão 1

* a classe Rastros passou a ter dois atributos: first e second, onde se registam os tokens dos jogadores, o que abre o jogo e do outro, respectivamente.
* foi corrigido o método utility (importante para a coevolução genérica)
* a coevolução para jogos genéricos passou a ser um extra com bónus de 0.25 e para a cotação de 1.5 pedimos apenas a coevolução para o Rastros especificamente.
* o enunciado foi melhorado, ficando mais claro os objectivos, a importância e conteúdo do relatório.